In [8]:
# Import necessary modules
import os
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv  # To load environment variables from the .env file

In [ ]:
# Load environment variables from .env
load_dotenv()

In [ ]:
# Read the API key from the environment variables
ALPHAVANTAGE_KEY = os.getenv('ALPHAVANTAGE_KEY')
if ALPHAVANTAGE_KEY is None:
    raise Exception("ALPHAVANTAGE_KEY not set in the .env file.")

In [ ]:
# Define a function to fetch daily adjusted time series data from AlphaVantage API for a given symbol
def get_daily_prices(symbol):
    """
    Fetches daily adjusted time series data for the provided symbol from AlphaVantage API.
    
    Parameters:
        symbol (str): The stock symbol to fetch data for (e.g., 'AAPL').
        
    Returns:
        pd.DataFrame: DataFrame containing the time series data with dates and prices.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'outputsize': 'compact',  # 'compact' returns the latest 100 data points; use 'full' for full history
        'apikey': ALPHAVANTAGE_KEY
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()